In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


base_dir = os.path.abspath(r"C:\eyesDetection\conjuctivies")
train_dir = os.path.join(base_dir, r"C:\eyesDetection\conjuctivies\trainEye")
val_dir = os.path.join(base_dir, r"C:\eyesDetection\conjuctivies\valEye")


IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32


train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=IMAGE_SIZE,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode="binary")  # Binary classification

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=IMAGE_SIZE,
                                                batch_size=BATCH_SIZE,
                                                class_mode="binary")

# CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])


model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])


history = model.fit(train_generator, validation_data=val_generator, epochs=10)

model.save("cow_eye.h5")

print("✅ Model trained and saved successfully!")


import cv2
import numpy as np
import tensorflow as tf


model = tf.keras.models.load_model("cow_conjunctivitis_model.h5")

def detect_conjunctivitis(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("❌ Error: Image not found!")
        return None


    image = cv2.resize(image, (224, 224))
    image = image / 255.0
    image = np.expand_dims(image, axis=0)


    prediction = model.predict(image)[0][0]

    if prediction > 0.545:
        print("Conjunctivitis Detected!")
    else:
        print("Cow is Healthy!")

test_image = r"C:\eyesDetection\conjuctivies\testEye\conj (62).jpeg"

detect_conjunctivitis(test_image)


Found 187 images belonging to 2 classes.
Found 26 images belonging to 2 classes.


C:\Users\Sagar Panchal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.5056 - loss: 1.1995 - val_accuracy: 0.4615 - val_loss: 0.6973
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.6255 - loss: 0.6816 - val_accuracy: 0.5385 - val_loss: 0.7017
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.5147 - loss: 0.7054 - val_accuracy: 0.4615 - val_loss: 0.7011
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.4702 - loss: 0.6995 - val_accuracy: 0.3077 - val_loss: 0.6975
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.5319 - loss: 0.6914 - val_accuracy: 0.4231 - val_loss: 0.7081
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5338 - loss: 0.6989 - val_accuracy: 0.3462 - val_loss: 0.7106
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5861 - loss: 0.6834 - val_accuracy: 0.4231 - val_loss: 0.7199
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5629 - loss: 0.6838 - val_accuracy: 0.3846 - val_loss: 0.7263
Epoch 9/10
6/6 

✅ Model trained and saved successfully!


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step
Cow is Healthy!


In [5]:
import tensorflow as tf

# Load the Keras model
model = tf.keras.models.load_model("cow_eye.h5")

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# (Optional) Enable optimizations for smaller model size (recommended for mobile)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save the .tflite model
with open("cow_eye.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ TFLite model saved as cow_eye.tflite")


INFO:tensorflow:Assets written to: C:\Users\SAGARP~1\AppData\Local\Temp\tmpjmz4k6gi\assets


INFO:tensorflow:Assets written to: C:\Users\SAGARP~1\AppData\Local\Temp\tmpjmz4k6gi\assets


Saved artifact at 'C:\Users\SAGARP~1\AppData\Local\Temp\tmpjmz4k6gi'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  3094794304400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3094794305168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3094810513616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3094810517840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3094794303056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3093659265616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3094794303632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3093659267344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3094810517456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  3094810517264: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ T

In [2]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import json # Import json to save class indices

# Set dataset paths
# IMPORTANT: Adjust these paths to where your eye images are located
base_dir = r"C:\eyesDetection\conjuctivies"
train_dir = os.path.join(base_dir, r"trainEye") # Assuming trainEye is directly under base_dir
val_dir = os.path.join(base_dir, r"valEye")     # Assuming valEye is directly under base_dir

IMAGE_SIZE = (224, 224) # Ensure this matches your model's input_shape
BATCH_SIZE = 32
EPOCHS = 10 # Define epochs as a variable

# Data preprocessing with augmentation for training
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

print(f"Loading training images from: {train_dir}")
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=IMAGE_SIZE,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode="binary") # Binary classification

print(f"Loading validation images from: {val_dir}")
val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=IMAGE_SIZE,
                                                batch_size=BATCH_SIZE,
                                                class_mode="binary")

# Get the class indices for reference (e.g., {'healthy': 0, 'conjunctivitis': 1})
health_class_indices = train_generator.class_indices
print("Health Class labels (index mapping):", health_class_indices)

# --- SAVE CLASS INDICES (Highly Recommended for clarity) ---
model_dir = "models" # Define a directory for models and related files
os.makedirs(model_dir, exist_ok=True) # Ensure this directory exists

with open(os.path.join(model_dir, "conjunctivitis_class_indices.json"), "w") as f:
    json.dump(health_class_indices, f)
print(f"✅ Class indices saved to {os.path.join(model_dir, 'conjunctivitis_class_indices.json')}")

# CNN Model
model_health = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid") # Output: one node for binary classification
])

model_health.compile(optimizer="adam",
                     loss="binary_crossentropy",
                     metrics=["accuracy"])

model_health.summary() # Print model summary

# Train the model
print(f"\nStarting model training for {EPOCHS} epochs...")
history_health = model_health.fit(train_generator, validation_data=val_generator, epochs=EPOCHS)

# Save the model
model_health.save(os.path.join(model_dir, "cow_eye.h5")) # Consistent naming
print(f"✅ Model trained and saved as {os.path.join(model_dir, 'cow_eye.h5')}")

print("\nTraining complete. You can now use 'cow_eye.h5' and 'conjunctivitis_class_indices.json' in your web app.")

# --- Optional: Test the reloaded model for a single prediction ---

print("\n--- Testing reloaded conjunctivitis model ---")
# Reload the saved model for prediction
model_reloaded_health = tf.keras.models.load_model(os.path.join(model_dir, "cow_eye.h5"))

# Load class indices
loaded_health_class_indices = {}
with open(os.path.join(model_dir, "conjunctivitis_class_indices.json"), "r") as f:
    loaded_health_class_indices = json.load(f)

# Reverse mapping for display
health_class_labels_reloaded = [None] * len(loaded_health_class_indices)
for label, index in loaded_health_class_indices.items():
    health_class_labels_reloaded[index] = label

def detect_conjunctivitis_test(image_path_test):
    try:
        # Load the image
        img = tf.keras.preprocessing.image.load_img(image_path_test, target_size=IMAGE_SIZE)
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        prediction = model_reloaded_health.predict(img_array)[0][0]

        # Use your specific threshold
        threshold = 0.545
        
        # Determine status based on the prediction and class indices
        if health_class_indices.get('conjunctivitis') is not None and health_class_indices['conjunctivitis'] == 1:
            # If 1 is conjunctivitis
            if prediction >= threshold:
                status = health_class_labels_reloaded[1] # e.g., 'conjunctivitis'
            else:
                status = health_class_labels_reloaded[0] # e.g., 'healthy'
        else: # Assuming 0 is conjunctivitis
            if prediction < threshold:
                status = health_class_labels_reloaded[0] # e.g., 'conjunctivitis'
            else:
                status = health_class_labels_reloaded[1] # e.g., 'healthy'


        print(f"📷 Test Image: {image_path_test}")
        print(f"👁️‍🗨️ Prediction Score: {prediction:.4f}")
        print(f"⚕️ Health Status: {status} (Confidence: {abs(prediction - (1-threshold))*100:.2f}%)") # Simplified confidence
    except Exception as e:
        print(f"Error during test prediction for {image_path_test}: {e}")

# IMPORTANT: Change this path to a real eye image for testing
test_image_eye = r"C:\eyesDetection\conjuctivies\testEye\conj (62).jpeg"
detect_conjunctivitis_test(test_image_eye)


Loading training images from: C:\eyesDetection\conjuctivies\trainEye
Found 187 images belonging to 2 classes.
Loading validation images from: C:\eyesDetection\conjuctivies\valEye
Found 26 images belonging to 2 classes.
Health Class labels (index mapping): {'conjuctivies': 0, 'healty': 1}
✅ Class indices saved to models\conjunctivitis_class_indices.json


C:\Users\Sagar Panchal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      44,302,848 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 44,396,609 (169.36 MB)

...

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4637 - loss: 1.9690

C:\Users\Sagar Panchal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 0.4639 - loss: 1.9330 - val_accuracy: 0.4615 - val_loss: 0.6997
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5152 - loss: 0.7091 - val_accuracy: 0.5000 - val_loss: 0.6935
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5241 - loss: 0.6943 - val_accuracy: 0.4615 - val_loss: 0.6962
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5209 - loss: 0.6902 - val_accuracy: 0.4615 - val_loss: 0.7116
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.4519 - loss: 0.7025 - val_accuracy: 0.4615 - val_loss: 0.7045
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5365 - loss: 0.6882 - val_accuracy: 0.4615 - val_loss: 0.7114
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5041 - loss: 0.6967 - val_accuracy: 0.4615 - val_loss: 0.7267
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5402 - loss: 0.6826 - val_accuracy: 0.5000 - val_loss: 0.7348
Epoch 9/10
6/6 ━━━━

✅ Model trained and saved as models\cow_eye.h5

Training complete. You can now use 'cow_eye.h5' and 'conjunctivitis_class_indices.json' in your web app.

--- Testing reloaded conjunctivitis model ---


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step
📷 Test Image: C:\eyesDetection\conjuctivies\testEye\conj (62).jpeg
👁️‍🗨️ Prediction Score: 0.6440
⚕️ Health Status: healty (Confidence: 18.90%)
